In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA

from pyLDAvis import sklearn as sklearn_lda
import pyLDAvis

import json

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics

In [3]:
contractions = ['aren', 'can', 'couldn', 'don', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'mustn', 'shan', 'shouldn',
                'wasn', 'weren', 'wouldn', 't',
               'he', 'i', 'she', 'they', 'we', 'you', 'd',
               'he', 'here', 'how', 'it', 'let', 'that', 'there', 'what', 'when', 'where', 'who', 'why', 's',
               'i', 'he', 'she', 'they', 'we', 'you', 'll',
               'i', 'm',
               'i', 'must', 'they', 'we', 'you', 've',
               'they', 'we', 'you', 're']
others = ['a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', 'arent', 'as',
              'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'cant', 'cannot', 
              'could', 'couldnt', 'did', 'didnt', 'do', 'does', 'doesnt', 'doing', 'dont', 'down', 'during', 'each', 'few',
              'for', 'from', 'further', 'had', 'hadnt', 'has', 'hasnt', 'have', 'havent', 'having', 'he', 'hed', 'hes',
              'her', 'here', 'heres', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'hows', 'i', 'id', 'im', 'ive',
              'if', 'in', 'into', 'is', 'isnt', 'it', 'its', 'itself', 'lets', 'me', 'more', 'most', 'mustnt', 'my', 
              'myself', 'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'ought', 'our', 'ours',
              'ourselves', 'out', 'over', 'own', 'same', 'shant', 'she', 'shes', 'should', 'shouldnt', 'so', 'some',
              'such', 'than', 'that', 'thats', 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there',
              'theres', 'these', 'they', 'theyd', 'theyll', 'theyre', 'theyve', 'this', 'those', 'through', 'to',
              'too', 'under', 'until', 'up', 'very', 'was', 'wasnt', 'we', 'were', 'weve', 'were', 'werent', 'what',
              'whats', 'when', 'whens', 'where', 'wheres', 'which', 'while', 'who', 'whos', 'whom', 'why', 'whys', 'with', 
              'wont', 'would', 'wouldnt', 'you', 'youd', 'youll', 'youre', 'youve', 'your', 'yours', 'yourself',
              'yourselves', 'm', 'f', 'tl', 'dr', 'tldr']
stop_words = list(set(contractions + others))

In [4]:
# df = pd.read_pickle('wsb_text.pkl')
df = pd.read_pickle('relationships_text.pkl')

In [5]:
cv = CountVectorizer(stop_words=stop_words)
word_counts = cv.fit_transform(df.text)
words = cv.get_feature_names()

In [6]:
def runLDA(N, filename):
    lda = LDA(n_components=N, n_jobs=-1, random_state=42)
    document_topic_distribution = lda.fit_transform(word_counts)
    
    n_top_words = 20
    
    with open(f"results/{filename}_topics.txt", "a") as outfile:
        print(f'Number of topics: {N}', file=outfile)
        for topic, comp in enumerate(lda.components_): 
            word_idx = np.argsort(comp)[::-1][:n_top_words]
            topic_words = [words[i] for i in word_idx]
            topic_weights = comp[word_idx] / np.linalg.norm(comp[word_idx])

            print(f'Topic {topic}: {" ".join(topic_words)} ({" ".join([str(x)[0:5] for x in topic_weights])})', 
                  file=outfile)
        print('', file=outfile)

    df['topic'] = -1

    for idx, document_topic in enumerate(document_topic_distribution):
        df.loc[idx, 'topic'] = document_topic.argmax()
        
    df.topic.to_json(f'results/{filename}_topics_{N}.json', orient='values')
        
    vectorizer = TfidfVectorizer(sublinear_tf=True)
    X = vectorizer.fit_transform(df.text)
    y = df.topic

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
        
    classifier = Pipeline(
        [('feature_selection', SelectFromModel(LinearSVC(penalty="l1", dual=False,tol=1e-3))),
         ('classification', LinearSVC(penalty="l2"))])
    
    classifier.fit(X_train, y_train)
    pred = classifier.predict(X_test)
    
    with open(f"results/{filename}_topics.txt", "a") as outfile:
        print(metrics.classification_report(y_test, pred), file=outfile)

In [ ]:
for i in range(5, 30, 5):
    runLDA(i, 'relationships')

In [7]:
for i in range(2, 10, 2):
    runLDA(i, 'relationships')

/home/chris/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/chris/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
/home/chris/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
